In [1]:
import sys
sys.path.append("../utils/") 
from utilities import performance_metrics
import time

from src.core import Term, Atom
from src.ilp import Language_Frame, Program_Template, Rule_Template
from src.dilp import DILP
import tensorflow as tf
import numpy as np
from src.utils import process_file, process_dir, create_table, output_rules, test_rule, train, test
from src.generate_template import create_templates
import duckdb
tf.random.set_seed(1000)
np.random.seed(1000)


2024-06-16 17:44:00.681355: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 17:44:01.992126: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
con = duckdb.connect(':memory:')
# enable automatic query parallelization
con.execute("PRAGMA threads=2")
# enable caching of parquet metadata
con.execute("PRAGMA enable_object_cache")

In [3]:
term_x_0 = Term(True, 'X_0')
term_x_1 = Term(True, 'X_1')
input_dir = "../examples/simple-transitivity-fraud-rule/"
input_table = create_table(con, input_dir)
print(f'ratio of positives %f' % (100*input_table['Fraud'].sum()/len(input_table)))
target, p_e, constants, B, P, N = process_dir(input_dir)

ratio of positives 50.000000
39
end of facts processing
5
end of positive examples processing
21
end negative examples processing
data is in order


## Templates to describe Rules to describe:


In [4]:
print(p_e)

[Fraud->orig->destination(X_0,X_1), Transfer->orig->destination(X_0,X_1)]


In [8]:
print(target)

Fraud_chain->orig->destination(X_0,X_1)


In [24]:
p_a

[]

In [5]:
p_a, rules = create_templates(p_e, target, term_x_0)
rules[target] = (Rule_Template(v=0, allow_intensional=True), Rule_Template(v=1, allow_intensional=True))

language_frame = Language_Frame(target, p_e, constants)
program_template = Program_Template(p_a, rules, T=6)

In [13]:
rules[target][0].v, rules[target][0].allow_intensional

(0, False)

In [14]:
rules[target][1].v, rules[target][1].allow_intensional

(1, True)

In [27]:
print("DILP initialisation")
dilp = DILP(language_frame, B, P, N, program_template, allow_target_recursion=True)
start_time = time.time()
dilp.train()
finish_time = time.time()
print("execution time %d" % (finish_time - start_time))

DILP initialisation
Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete


KeyboardInterrupt: 